# LANISTR Cloud Deployment Tutorial

This notebook shows you how to deploy LANISTR training jobs to Google Cloud Vertex AI using the actual project configurations.

## What is LANISTR?

LANISTR (LANguage, Image, and STRuctured data) is a multimodal learning framework that can handle:

- **Unstructured Data**: Text (reports, descriptions) and Images (X-rays, product photos)
- **Structured Data**: Tabular data (patient demographics, product features) and Time-series data (vital signs, sensor readings)

### Key Features:

1. **Masking-based Training**: Uses masking strategies across both unimodal and multimodal levels
2. **Cross-modal Learning**: Introduces similarity-based multimodal masking loss for learning cross-modal relations
3. **Missing Modality Robustness**: Works effectively even when some modalities are missing from samples
4. **Cloud-Native**: Designed for deployment on Google Cloud Vertex AI

### Supported Datasets:

- **MIMIC-IV**: Healthcare dataset with chest X-rays, clinical notes, and patient vitals
- **Amazon Product Review**: Retail dataset with product images, reviews, and metadata

## Why Cloud Deployment?

1. **No Local GPU Required**: Training happens on Google Cloud's powerful GPU instances
2. **Scalability**: Easy to scale from single GPU to multi-GPU or distributed training
3. **Cost Effective**: Pay only for compute time used
4. **Reproducible**: Consistent environment across different machines
5. **Managed Infrastructure**: No need to manage CUDA, drivers, or hardware

Let's get started with cloud deployment!

## Prerequisites Setup

First, let's set up the minimal local environment needed for cloud submission.

In [1]:
# Import necessary libraries for cloud deployment
import os
import sys
import json
import yaml
from pathlib import Path
from datetime import datetime

# Add the lanistr directory to the path
sys.path.append('./lanistr')

# Check if we're in the right directory
print(f"Current working directory: {os.getcwd()}")
print(f"LANISTR directory exists: {os.path.exists('./lanistr')}")

# Check if we have the cloud requirements installed
try:
    import omegaconf
    print("✅ Core utilities imported successfully")
except ImportError as e:
    print(f"❌ Missing core utilities: {e}")
    print("Run: pip install -r requirements-cloud.txt")

# Check Google Cloud setup
try:
    from google.cloud import aiplatform
    from google.cloud import storage
    print("✅ Google Cloud libraries imported successfully")
except ImportError as e:
    print(f"❌ Missing Google Cloud libraries: {e}")
    print("Run: pip install -r requirements-cloud.txt")

Current working directory: /Users/stuartgano/lanistr
LANISTR directory exists: True
✅ Core utilities imported successfully
✅ Google Cloud libraries imported successfully


## Google Cloud Setup

Before we can submit jobs to Vertex AI, we need to set up Google Cloud authentication and configuration.

In [2]:
# Google Cloud configuration - UPDATE THESE WITH YOUR ACTUAL VALUES
PROJECT_ID = "mgm-digitalconcierge"  # Replace with your actual project ID
REGION = "us-central1"  # Choose your preferred region
BUCKET_NAME = "lanistr-mgm-digitalconcierge-data"  # Replace with your bucket name

# Initialize Vertex AI
try:
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print(f"✅ Vertex AI initialized for project: {PROJECT_ID}")
    print(f"✅ Region: {REGION}")
except Exception as e:
    print(f"❌ Failed to initialize Vertex AI: {e}")
    print("\nTo set up Google Cloud:")
    print("1. Install Google Cloud SDK: https://cloud.google.com/sdk/docs/install")
    print("2. Run: gcloud auth login")
    print("3. Run: gcloud config set project YOUR_PROJECT_ID")
    print("4. Enable Vertex AI API: gcloud services enable aiplatform.googleapis.com")

# Test bucket access
try:
    storage_client = storage.Client(project=PROJECT_ID)
    bucket = storage_client.bucket(BUCKET_NAME)
    print(f"✅ Successfully connected to bucket: {BUCKET_NAME}")
except Exception as e:
    print(f"❌ Failed to access bucket: {e}")
    print("\nTo create a bucket:")
    print(f"gcloud storage buckets create gs://{BUCKET_NAME} --location={REGION}")

✅ Vertex AI initialized for project: mgm-digitalconcierge
✅ Region: us-central1
✅ Successfully connected to bucket: lanistr-mgm-digitalconcierge-data


## Understanding LANISTR Project Structure

Let's examine the actual project structure and configurations.

In [ ]:
# List available configurations
print("Available Vertex AI configurations:")
vertex_configs = [
    'vertex_ai_configs/mimic_pretrain_vertex.yaml',
    'vertex_ai_configs/mimic_finetune_vertex.yaml',
    'vertex_ai_configs/amazon_pretrain_vertex.yaml'
]
for config in vertex_configs:
    if os.path.exists(config):
        print(f"  ✅ {config}")
    else:
        print(f"  ❌ {config} (not found)")

print("\nAvailable LANISTR configurations:")
lanistr_configs = [
    'lanistr/configs/mimic_pretrain.yaml',
    'lanistr/configs/mimic_finetune.yaml',
    'lanistr/configs/amazon_finetune_beauty.yaml',
    'lanistr/configs/amazon_finetune_fashion.yaml',
    'lanistr/configs/amazon_pretrain_office.yaml'
]
for config in lanistr_configs:
    if os.path.exists(config):
        print(f"  ✅ {config}")
    else:
        print(f"  ❌ {config} (not found)")

# Load and display a sample configuration
sample_config_path = 'lanistr/configs/mimic_pretrain.yaml'
if os.path.exists(sample_config_path):
    with open(sample_config_path, 'r') as f:
        config = yaml.safe_load(f)
    
    print("\nSample Configuration Structure:")
    print(f"Dataset: {config.get('dataset_name', 'N/A')}")
    print(f"Task: {config.get('task', 'N/A')}")
    print(f"Modalities enabled:")
    print(f"  - Image: {config.get('image', False)}")
    print(f"  - Text: {config.get('text', False)}")
    print(f"  - Tabular: {config.get('tab', False)}")
    print(f"  - Time-series: {config.get('time', False)}")
    print(f"\nTraining parameters:")
    print(f"  - Batch size: {config.get('train_batch_size', 'N/A')}")
    print(f"  - Learning rate: {config.get('optimizer', {}).get('learning_rate', 'N/A')}")
    print(f"  - Epochs: {config.get('scheduler', {}).get('num_epochs', 'N/A')}")
    print(f"\nData paths:")
    print(f"  - Root data: {config.get('root_data_dir', 'N/A')}")
    print(f"  - Output: {config.get('output_dir', 'N/A')}")

## Data Preparation for Cloud

Let's prepare and upload the actual data from your project to Google Cloud Storage.

In [3]:
# Data preparation functions
def upload_data_to_gcs(local_path, gcs_path):
    """Upload data to Google Cloud Storage"""
    try:
        storage_client = storage.Client(project=PROJECT_ID)
        bucket = storage_client.bucket(BUCKET_NAME)
        
        if os.path.isfile(local_path):
            # Upload single file
            blob = bucket.blob(gcs_path)
            blob.upload_from_filename(local_path)
            print(f"✅ Uploaded {local_path} to gs://{BUCKET_NAME}/{gcs_path}")
        elif os.path.isdir(local_path):
            # Upload directory recursively
            for root, dirs, files in os.walk(local_path):
                for file in files:
                    local_file_path = os.path.join(root, file)
                    relative_path = os.path.relpath(local_file_path, local_path)
                    gcs_file_path = os.path.join(gcs_path, relative_path)
                    
                    blob = bucket.blob(gcs_file_path)
                    blob.upload_from_filename(local_file_path)
                    print(f"✅ Uploaded {local_file_path} to gs://{BUCKET_NAME}/{gcs_file_path}")
        
        return f"gs://{BUCKET_NAME}/{gcs_path}"
    except Exception as e:
        print(f"❌ Failed to upload {local_path}: {e}")
        return None

# Actual data paths from your project
print("Actual Data Paths in Your Project:")
data_paths = [
    ('./data/mimic-iv/', 'lanistr-data/mimic-iv/'),
    ('./lanistr/configs/', 'lanistr-configs/'),
    ('./vertex_ai_configs/', 'vertex-ai-configs/')
]

for local_path, gcs_path in data_paths:
    if os.path.exists(local_path):
        print(f"✅ {local_path} -> gs://{BUCKET_NAME}/{gcs_path}")
    else:
        print(f"❌ {local_path} (not found)")

# Example: Upload MIMIC-IV data
print("\nTo upload MIMIC-IV data:")
print("upload_data_to_gcs('./data/mimic-iv/', 'lanistr-data/mimic-iv/')")
print("\nTo upload configurations:")
print("upload_data_to_gcs('./lanistr/configs/', 'lanistr-configs/')")
print("upload_data_to_gcs('./vertex_ai_configs/', 'vertex-ai-configs/')")

Actual Data Paths in Your Project:
✅ ./data/mimic-iv/ -> gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/
✅ ./lanistr/configs/ -> gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/
✅ ./vertex_ai_configs/ -> gs://lanistr-mgm-digitalconcierge-data/vertex-ai-configs/

To upload MIMIC-IV data:
upload_data_to_gcs('./data/mimic-iv/', 'lanistr-data/mimic-iv/')

To upload configurations:
upload_data_to_gcs('./lanistr/configs/', 'lanistr-configs/')
upload_data_to_gcs('./vertex_ai_configs/', 'vertex-ai-configs/')


## Building the Container with Cloud Build

Instead of building the container locally, we'll use Google Cloud Build which is faster and more reliable.

### Option 1: Use the Automated Script (Recommended)

We've created an automated script that handles everything:

1. **Update the script**: Edit `trigger_cloud_build.sh` and set your PROJECT_ID
2. **Run the script**: `./trigger_cloud_build.sh`

The script will:
- ✅ Check prerequisites (gcloud, authentication)
- ✅ Enable required APIs
- ✅ Trigger Cloud Build
- ✅ Stream build logs to your terminal
- ✅ Provide build status and container URI

### Option 2: Manual Cloud Build

If you prefer manual control:

```bash
# 1. Enable required APIs
gcloud services enable cloudbuild.googleapis.com
gcloud services enable containerregistry.googleapis.com
gcloud services enable aiplatform.googleapis.com

# 2. Trigger Cloud Build
gcloud builds submit --config=cloudbuild.yaml .

# 3. Monitor build progress
gcloud builds list --limit=5
```

### Why Cloud Build?

- **No Local Resources**: Builds happen on Google Cloud's powerful infrastructure
- **Faster Builds**: High-CPU machines with more memory and disk space
- **Consistent Environment**: Same build environment every time
- **No Docker Installation**: No need to install Docker locally
- **Automatic Pushing**: Container is automatically pushed to Container Registry

### Container Details

- **Base Image**: NVIDIA CUDA 11.3 with cuDNN8 on Ubuntu 20.04
- **Python**: 3.8
- **PyTorch**: 2.0+ with CUDA support
- **Entry Point**: `lanistr/main_vertex_ai.py` (optimized for Vertex AI)
- **Security**: Non-root user for production safety

Let's check if you have the prerequisites and trigger the build:

In [ ]:
# Check Cloud Build prerequisites and trigger build
import subprocess
import os

def check_command(command, name):
    """Check if a command is available"""
    try:
        result = subprocess.run([command, '--version'], 
                               capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            version = result.stdout.strip().split('\n')[0]
            print(f"✅ {name}: {version}")
            return True
        else:
            print(f"❌ {name}: Not found")
            return False
    except (subprocess.TimeoutExpired, FileNotFoundError):
        print(f"❌ {name}: Not found")
        return False

print("Checking Cloud Build prerequisites:")
print("=" * 50)

# Check gcloud
gcloud_ok = check_command('gcloud', 'Google Cloud SDK')

# Check if we're authenticated
try:
    result = subprocess.run(['gcloud', 'auth', 'list', '--filter=status:ACTIVE', '--format=value(account)'], 
                           capture_output=True, text=True, timeout=10)
    if result.returncode == 0 and result.stdout.strip():
        print(f"✅ Google Cloud Auth: {result.stdout.strip()}")
        auth_ok = True
    else:
        print("❌ Google Cloud Auth: Not authenticated")
        auth_ok = False
except:
    print("❌ Google Cloud Auth: Error checking")
    auth_ok = False

# Check if cloudbuild.yaml exists
if os.path.exists('cloudbuild.yaml'):
    print("✅ cloudbuild.yaml: Found")
    cloudbuild_ok = True
else:
    print("❌ cloudbuild.yaml: Not found")
    cloudbuild_ok = False

# Check if trigger script exists
if os.path.exists('trigger_cloud_build.sh'):
    print("✅ trigger_cloud_build.sh: Found")
    script_ok = True
else:
    print("❌ trigger_cloud_build.sh: Not found")
    script_ok = False

print("\nSummary:")
if all([gcloud_ok, auth_ok, cloudbuild_ok, script_ok]):
    print("🎉 All prerequisites are ready! You can trigger Cloud Build.")
    print(f"\nTo build the container:")
    print(f"1. Update PROJECT_ID in trigger_cloud_build.sh")
    print(f"2. Run: ./trigger_cloud_build.sh")
    print(f"\nOr manually:")
    print(f"gcloud builds submit --config=cloudbuild.yaml .")
else:
    print("⚠️  Some prerequisites are missing. Please install them first.")
    if not gcloud_ok:
        print("  - Install Google Cloud SDK: https://cloud.google.com/sdk/docs/install")
    if not auth_ok:
        print("  - Run: gcloud auth login")
    if not cloudbuild_ok:
        print("  - cloudbuild.yaml should be in the project root")
    if not script_ok:
        print("  - trigger_cloud_build.sh should be in the project root")

# Show current project
try:
    result = subprocess.run(['gcloud', 'config', 'get-value', 'project'], 
                           capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        current_project = result.stdout.strip()
        print(f"\nCurrent Google Cloud Project: {current_project}")
        if current_project != PROJECT_ID:
            print(f"⚠️  Note: Current project ({current_project}) differs from PROJECT_ID ({PROJECT_ID})")
            print(f"   You may want to run: gcloud config set project {PROJECT_ID}")
except:
    print("\nCould not determine current Google Cloud project")

print("\n" + "="*60)
print("🚀 READY TO BUILD! 🚀")
print("="*60)
print(f"\nYour container will be available at:")
print(f"gcr.io/{PROJECT_ID}/lanistr:latest")
print(f"\nAfter building, update your notebook with this container URI.")

In [6]:
upload_data_to_gcs('./data/mimic-iv/', 'lanistr-data/mimic-iv/')
upload_data_to_gcs('./lanistr/configs/', 'lanistr-configs/')
upload_data_to_gcs('./vertex_ai_configs/', 'vertex-ai-configs/')


/Users/stuartgano/lanistr/venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Uploaded ./data/mimic-iv/.DS_Store to gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/.DS_Store
✅ Uploaded ./data/mimic-iv/mimic-iv.jsonl to gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/mimic-iv.jsonl
✅ Uploaded ./data/mimic-iv/timeseries/patient_p16161191_vitals.csv to gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/timeseries/patient_p16161191_vitals.csv
✅ Uploaded ./data/mimic-iv/timeseries/patient_p38314794_vitals.csv to gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/timeseries/patient_p38314794_vitals.csv
✅ Uploaded ./data/mimic-iv/timeseries/patient_p27673588_vitals.csv to gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/timeseries/patient_p27673588_vitals.csv
✅ Uploaded ./data/mimic-iv/timeseries/patient_p78348834_vitals.csv to gs://lanistr-mgm-digitalconcierge-data/lanistr-data/mimic-iv/timeseries/patient_p78348834_vitals.csv
✅ Uploaded ./data/mimic-iv/timeseries/patient_p88088261_vitals.csv to gs://lanistr-m

/Users/stuartgano/lanistr/venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Uploaded ./lanistr/configs/mimic-iv_pretrain_local.yaml to gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/mimic-iv_pretrain_local.yaml
✅ Uploaded ./lanistr/configs/amazon_finetune_beauty.yaml to gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/amazon_finetune_beauty.yaml
✅ Uploaded ./lanistr/configs/mimic_finetune.yaml to gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/mimic_finetune.yaml
✅ Uploaded ./lanistr/configs/amazon_pretrain_office.yaml to gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/amazon_pretrain_office.yaml
✅ Uploaded ./lanistr/configs/mimic_pretrain.yaml to gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/mimic_pretrain.yaml
✅ Uploaded ./lanistr/configs/amazon_finetune_fashion.yaml to gs://lanistr-mgm-digitalconcierge-data/lanistr-configs/amazon_finetune_fashion.yaml


/Users/stuartgano/lanistr/venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Uploaded ./vertex_ai_configs/mimic_finetune_vertex.yaml to gs://lanistr-mgm-digitalconcierge-data/vertex-ai-configs/mimic_finetune_vertex.yaml
✅ Uploaded ./vertex_ai_configs/mimic_pretrain_vertex.yaml to gs://lanistr-mgm-digitalconcierge-data/vertex-ai-configs/mimic_pretrain_vertex.yaml
✅ Uploaded ./vertex_ai_configs/amazon_pretrain_vertex.yaml to gs://lanistr-mgm-digitalconcierge-data/vertex-ai-configs/amazon_pretrain_vertex.yaml


'gs://lanistr-mgm-digitalconcierge-data/vertex-ai-configs/'

In [ ]:
! gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=16mbYEIYB3H9qKlQ8XHzMjE15E0hTO&access_type=offline&code_challenge=GekRtzysKR1IQAEyLioOueacRj3GMjBJBr-q2KJykfY&code_challenge_method=S256


Credentials saved to file: [/Users/stuartgano/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Cannot add the project "831221572395" to ADC as the quota project because the account in ADC does not have the "serviceusage.services.use" permission on this project. You might receive a "quota_exceeded" or "API not enabled" error. Run 

In [19]:
! gcloud auth application-default set-quota-project mgm-digitalconcierge


Credentials saved to file: [/Users/stuartgano/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "mgm-digitalconcierge" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## Creating Vertex AI Training Jobs

Now let's create actual training job configurations using your project's Vertex AI configs.

In [21]:
# Function to create Vertex AI custom job
def create_vertex_ai_job(
    job_name,
    config_file,
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1
):
    """Create a Vertex AI custom training job using actual project configs"""
    
    # Container image for LANISTR (you'll need to build this)
    container_uri = f"gcr.io/mgm-digitalconcierge/lanistr:latest"
    
    # Job configuration
    job_config = {
        "display_name": job_name,
        "job_spec": {
            "worker_pool_specs": [
                {
                    "machine_spec": {
                        "machine_type": machine_type,
                        "accelerator_type": accelerator_type,
                        "accelerator_count": accelerator_count
                    },
                    "replica_count": 1,
                    "container_spec": {
                        "image_uri": container_uri,
                        "args": [
                            "python", "lanistr/main.py",
                            "--config", config_file
                        ]
                    }
                }
            ]
        }
    }
    
    return job_config

# Actual job configurations from your project
print("Available Training Jobs:")

# MIMIC-IV Pre-training
mimic_pretrain_config = create_vertex_ai_job(
    job_name="mimic-pretrain-001",
    config_file=f"gs://{BUCKET_NAME}/vertex-ai-configs/mimic_pretrain_vertex.yaml",
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1
)
print("\n1. MIMIC-IV Pre-training:")
print(json.dumps(mimic_pretrain_config, indent=2))

# MIMIC-IV Fine-tuning
mimic_finetune_config = create_vertex_ai_job(
    job_name="mimic-finetune-001",
    config_file=f"gs://{BUCKET_NAME}/vertex-ai-configs/mimic_finetune_vertex.yaml",
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1
)
print("\n2. MIMIC-IV Fine-tuning:")
print(json.dumps(mimic_finetune_config, indent=2))

# Amazon Pre-training
amazon_pretrain_config = create_vertex_ai_job(
    job_name="amazon-pretrain-001",
    config_file=f"gs://{BUCKET_NAME}/vertex-ai-configs/amazon_pretrain_vertex.yaml",
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1
)
print("\n3. Amazon Pre-training:")
print(json.dumps(amazon_pretrain_config, indent=2))

mimic_pretrain_config = create_vertex_ai_job(
    job_name="mimic-pretrain-001",
    config_file=f"gs://{BUCKET_NAME}/vertex-ai-configs/mimic_pretrain_vertex.yaml",
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_V100",
    accelerator_count=1
)
mimic_finetune_config = create_vertex_ai_job(
    job_name="mimic-finetune-001",
    config_file=f"gs://{BUCKET_NAME}/vertex-ai-configs/mimic_finetune_vertex.yaml",
    machine_type="n1-standard-4",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1
)


Available Training Jobs:

1. MIMIC-IV Pre-training:
{
  "display_name": "mimic-pretrain-001",
  "job_spec": {
    "worker_pool_specs": [
      {
        "machine_spec": {
          "machine_type": "n1-standard-8",
          "accelerator_type": "NVIDIA_TESLA_V100",
          "accelerator_count": 1
        },
        "replica_count": 1,
        "container_spec": {
          "image_uri": "gcr.io/mgm-digitalconcierge/lanistr:latest",
          "args": [
            "python",
            "lanistr/main.py",
            "--config",
            "gs://lanistr-mgm-digitalconcierge-data/vertex-ai-configs/mimic_pretrain_vertex.yaml"
          ]
        }
      }
    ]
  }
}

2. MIMIC-IV Fine-tuning:
{
  "display_name": "mimic-finetune-001",
  "job_spec": {
    "worker_pool_specs": [
      {
        "machine_spec": {
          "machine_type": "n1-standard-4",
          "accelerator_type": "NVIDIA_TESLA_T4",
          "accelerator_count": 1
        },
        "replica_count": 1,
        "container_s

In [20]:
! ./trigger_cloud_build.sh

🚀 LANISTR Cloud Build Trigger Script

Configuration:
  Project ID: mgm-digitalconcierge
  Region: us-central1

📋 Step 1: Checking prerequisites...
✅ Google Cloud SDK is installed
✅ Authenticated with Google Cloud
⚠️  Setting project to mgm-digitalconcierge
Reauthentication required.
Please enter your password:

## Submitting Training Jobs

Now let's submit the actual training jobs to Vertex AI.

In [28]:
# Function to submit training jobs
def submit_training_job(job_config):
    """Submit a training job to Vertex AI"""
    try:
        # Create the custom job
        job = aiplatform.CustomJob(
            display_name=job_config["display_name"],
            worker_pool_specs=job_config["job_spec"]["worker_pool_specs"],
            project=PROJECT_ID,
            location='us-central1',
            staging_bucket=f'gs://{BUCKET_NAME}'

        )
        
        # Submit the job
        job.run()
        
        print(f"✅ Successfully submitted job: {job_config['display_name']}")
        print(f"Job ID: {job.name}")
        print(f"Job URL: https://console.cloud.google.com/vertex-ai/training/custom-jobs/{job.name}")
        
        return job
        
    except Exception as e:
        print(f"❌ Failed to submit job: {e}")
        return None

# Submit actual jobs
print("Submitting Training Jobs:")
print("\nNote: Uncomment the lines below to actually submit jobs")

# # Submit MIMIC-IV pre-training job
mimic_pretrain_job = submit_training_job(mimic_pretrain_config)
# 
# # Submit MIMIC-IV fine-tuning job
# mimic_finetune_job = submit_training_job(mimic_finetune_config)
# 
# # Submit Amazon pre-training job
# amazon_pretrain_job = submit_training_job(amazon_pretrain_config)

print("\nTo submit jobs manually:")
print("1. Update PROJECT_ID, REGION, and BUCKET_NAME above")
print("2. Upload your data and configs to GCS")
print("3. Build and push your LANISTR container")
print("4. Uncomment the job submission lines above")
print("5. Run the cell")

Submitting Training Jobs:

Note: Uncomment the lines below to actually submit jobs
Creating CustomJob
CustomJob created. Resource name: projects/419850945193/locations/us-central1/customJobs/2698590297822068736
To use this CustomJob in another session:
custom_job = aiplatform.CustomJob.get('projects/419850945193/locations/us-central1/customJobs/2698590297822068736')
View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/2698590297822068736?project=419850945193
CustomJob projects/419850945193/locations/us-central1/customJobs/2698590297822068736 current state:
2
CustomJob projects/419850945193/locations/us-central1/customJobs/2698590297822068736 current state:
2
CustomJob projects/419850945193/locations/us-central1/customJobs/2698590297822068736 current state:
2
CustomJob projects/419850945193/locations/us-central1/customJobs/2698590297822068736 current state:
2
CustomJob projects/419850945193/locations/us-central1/customJobs/2698590297822068736 curr

## Complete Workflow Example

Let's put everything together in a complete workflow using your actual project structure.

In [ ]:
# Complete workflow function
def run_complete_workflow(
    dataset_name,
    task_type,
    job_name=None
):
    """Run a complete LANISTR training workflow on Vertex AI"""
    
    if job_name is None:
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        job_name = f"{dataset_name}-{task_type}-{timestamp}"
    
    print(f"🚀 Starting LANISTR {task_type} workflow for {dataset_name}")
    print(f"Job name: {job_name}")
    print("-" * 60)
    
    # Step 1: Upload data
    print("📤 Step 1: Uploading data...")
    if dataset_name == 'mimic-iv':
        data_gcs_path = upload_data_to_gcs('./data/mimic-iv/', 'lanistr-data/mimic-iv/')
    else:
        print(f"❌ Dataset {dataset_name} not supported yet")
        return None
    
    # Step 2: Upload configurations
    print("📤 Step 2: Uploading configurations...")
    config_gcs_path = upload_data_to_gcs('./vertex_ai_configs/', 'vertex-ai-configs/')
    
    # Step 3: Create job configuration
    print("⚙️  Step 3: Creating job configuration...")
    config_file = f"gs://{BUCKET_NAME}/vertex-ai-configs/{dataset_name}_{task_type}_vertex.yaml"
    job_config = create_vertex_ai_job(
        job_name=job_name,
        config_file=config_file
    )
    
    # Step 4: Submit job
    print("🚀 Step 4: Submitting training job...")
    job = submit_training_job(job_config)
    
    if job:
        print("\n✅ Workflow completed successfully!")
        print(f"Monitor your job at: https://console.cloud.google.com/vertex-ai/training/custom-jobs/{job.name}")
        return job
    else:
        print("\n❌ Workflow failed!")
        return None

# Example workflow calls
print("Complete Workflow Examples:")
print("\n1. MIMIC-IV Pre-training:")
print("run_complete_workflow('mimic-iv', 'pretrain')")

print("\n2. MIMIC-IV Fine-tuning:")
print("run_complete_workflow('mimic-iv', 'finetune')")

print("\nTo run a workflow:")
print("1. Update PROJECT_ID, REGION, and BUCKET_NAME above")
print("2. Build and push your LANISTR container to GCR")
print("3. Uncomment and run the workflow calls above")

## Next Steps

Now that you understand how to deploy LANISTR to the cloud, here are your next steps:

1. **Set up Google Cloud**: Complete the authentication and project setup
2. **Build LANISTR container**: Create a Docker container with LANISTR and push to GCR
3. **Upload your data**: Use the upload functions to get your data to GCS
4. **Submit training jobs**: Use the provided functions to start training
5. **Monitor and optimize**: Track job progress and optimize for cost/performance
6. **Download results**: Retrieve your trained models and logs

## Summary

This tutorial has shown you how to:

- Set up Google Cloud authentication and configuration
- Upload data and configurations to Google Cloud Storage
- Create and submit custom training jobs to Vertex AI using your actual project configs
- Monitor job progress and download results
- Optimize costs and troubleshoot common issues

LANISTR provides a powerful framework for multimodal learning that can be easily deployed to the cloud, allowing you to focus on your research and development without worrying about local hardware constraints.

Happy cloud training! ☁️🚀